### Download packages


In [ ]:
! pip install spacy
! python -m spacy download en_core_web_sm


import spacy
nlp = spacy.load("en_core_web_sm")


text = [
    "This is her book.",
    "Give the book back to her."
]


for t in text:
    doc = nlp(t)

    for tok in doc:
        print(tok.text, tok.dep_)

    print()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
This nsubj
is ROOT
her poss
book attr
. punct

Give ROOT
the det
book dobj
back advmod
to prep
her pobj
. punct



# **Part1**

In [ ]:
!pip install pandas numpy torch==2.0.1 torchtext==0.15.2 nltk scikit-learn

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv("IMDB Dataset.csv")
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
tokenizer = get_tokenizer("basic_english")
stop_words = set(stopwords.words("english"))

def tokenize(text):
    tokens = tokenizer(text.lower())
    return [t for t in tokens if t not in stop_words]

df['tokens'] = df['review'].apply(tokenize)

In [ ]:
from collections import Counter
import torch
from torchtext.vocab import vocab

# Assuming df['tokens'] is a column of tokenized lists
counter = Counter()
for tokens in df['tokens']:
    counter.update(tokens)

# Use `vocab(...)` directly — no need for torchtext.vocab.vocab
vocab_obj = vocab(counter, specials=["<unk>", "<pad>"])
vocab_obj.set_default_index(vocab_obj["<unk>"])

# Example usage
print(vocab_obj["<pad>"])
print(vocab_obj["some_word"])

1
0


In [ ]:
# Numericalize function
def numericalize(tokens):
    return [vocab_obj[token] for token in tokens]

df['numerical'] = df['tokens'].apply(numericalize)

# Pad sequences
MAX_LEN = 300

def pad_input(seq):
    seq = seq[:MAX_LEN]
    return torch.tensor(seq + [vocab_obj["<pad>"]] * (MAX_LEN - len(seq)))

df['padded'] = df['numerical'].apply(pad_input)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [ ]:
import torch
from sklearn.model_selection import train_test_split

# Inputs
X = torch.stack(df['padded'].tolist())
y = torch.tensor(df['sentiment'].tolist(), dtype=torch.long)

# Convert to list for train_test_split
X_list = X.tolist()
y_list = y.tolist()

# Train-test split
X_train_list, X_test_list, y_train_list, y_test_list = train_test_split(X_list, y_list, test_size=0.2)

# Convert back to tensors
X_train = torch.tensor(X_train_list, dtype=torch.long)
X_test = torch.tensor(X_test_list, dtype=torch.long)
y_train = torch.tensor(y_train_list, dtype=torch.long)
y_test = torch.tensor(y_test_list, dtype=torch.long)

# Create datasets and dataloaders
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

In [ ]:
import torch
import torch.nn as nn

class RNNClassifier(nn.Module):
    def __init__(self, embedding_matrix):
        super(RNNClassifier, self).__init__()
        embedding_dim = embedding_matrix.shape[1]
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        embedded = self.embedding(x)                           # [batch_size, seq_len, emb_dim]
        output, hidden = self.rnn(embedded)                    # hidden: [1, batch_size, hidden_dim]
        out = self.fc(hidden.squeeze(0))                       # [batch_size, 1]
        return torch.sigmoid(out).squeeze(1)                   # [batch_size]

In [ ]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_matrix):
        super(LSTMClassifier, self).__init__()
        embedding_dim = embedding_matrix.shape[1]  # safer than hardcoding 100
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        embedded = self.embedding(x)                      # [batch_size, seq_len, emb_dim]
        _, (hidden, _) = self.lstm(embedded)              # hidden: [1, batch_size, 128]
        out = self.fc(hidden.squeeze(0))                  # [batch_size, 1]
        return torch.sigmoid(out).squeeze(1)              # [batch_size]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_model(model, loader, epochs=5):
    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device).float()

            optimizer.zero_grad()
            outputs = model(xb)  # shape: [batch_size]
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Calculate accuracy during training (optional)
            preds = (outputs >= 0.5).float()
            correct += (preds == yb).sum().item()
            total += yb.size(0)

        avg_loss = total_loss / len(loader)
        accuracy = correct / total * 100
        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")

In [ ]:
def evaluate(model, loader):
    model = model.to(device)
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device).float()  # Ensure yb is float for comparison
            outputs = model(xb)
            preds = (outputs >= 0.5).float()
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# -------------------------
# Device configuration
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -------------------------
# Dummy data (replace with real data)
# -------------------------
# Assume each input sequence is 100-dimensional and of length 50
vocab_size = 5000
embedding_dim = 100
sequence_length = 50
num_samples = 1000

X = torch.randint(0, vocab_size, (num_samples, sequence_length))
y = torch.randint(0, 2, (num_samples,)).float()

# -------------------------
# Embedding matrix (random for demo)
# Replace with pretrained like GloVe if available
# -------------------------
embedding_matrix = torch.randn(vocab_size, embedding_dim)

# -------------------------
# Dataset and DataLoader
# -------------------------
dataset = TensorDataset(X, y)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# -------------------------
# Model Definitions
# -------------------------
class RNNClassifier(nn.Module):
    def __init__(self, embedding_matrix):
        super(RNNClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.rnn = nn.RNN(embedding_dim, 128, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        return torch.sigmoid(self.fc(hidden.squeeze(0)))

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_matrix):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_dim, 128, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        return torch.sigmoid(self.fc(hidden.squeeze(0)))

# -------------------------
# Training function
# -------------------------
def train_model(model, loader, epochs=5):
    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device).float()
            optimizer.zero_grad()
            outputs = model(xb).squeeze()
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

# -------------------------
# Evaluation function
# -------------------------
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device).float()
            preds = (model(xb).squeeze() > 0.5).float()
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    accuracy = 100 * correct / total if total != 0 else 0
    print(f"Accuracy: {accuracy:.2f}%")

# -------------------------
# Run RNN
# -------------------------
print("Training RNN...")
rnn_model = RNNClassifier(embedding_matrix)
train_model(rnn_model, train_loader)
evaluate(rnn_model, test_loader)

# -------------------------
# Run LSTM
# -------------------------
print("\nTraining LSTM...")
lstm_model = LSTMClassifier(embedding_matrix)
train_model(lstm_model, train_loader)
evaluate(lstm_model, test_loader)


Training RNN...
Epoch 1, Loss: 0.7032
Epoch 2, Loss: 0.6154
Epoch 3, Loss: 0.4966
Epoch 4, Loss: 0.3175
Epoch 5, Loss: 0.1441
Accuracy: 49.00%

Training LSTM...
Epoch 1, Loss: 0.6913
Epoch 2, Loss: 0.6488
Epoch 3, Loss: 0.5712
Epoch 4, Loss: 0.4147
Epoch 5, Loss: 0.2134
Accuracy: 53.00%


# **Part2**

In [ ]:
import re
import csv

# Month name to number mapping
MONTHS = {
    'january': '01', 'february': '02', 'march': '03', 'april': '04',
    'may': '05', 'june': '06', 'july': '07', 'august': '08',
    'september': '09', 'october': '10', 'november': '11', 'december': '12'
}

# Regex patterns for different date formats
DATE_PATTERNS = [
    r'(\d{1,2})(st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)[,]?\s+(\d{4})',
    r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2})(st|nd|rd|th)?[,]?\s+(\d{4})',
    r'(\d{1,2})/(\d{1,2})/(\d{4})',
    r'(\d{1,2})-(\d{1,2})-(\d{4})'
]

# Function to extract date from text
def extract_date(text):
    for pattern in DATE_PATTERNS:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            groups = match.groups()
            if len(groups) == 4:
                # Case: 21st June 2024 or June 21st 2024
                if groups[0].isdigit():  # 21st June 2024
                    day = int(groups[0])
                    month = MONTHS[groups[2].lower()]
                    year = int(groups[3])
                else:  # June 21st 2024
                    day = int(groups[1])
                    month = MONTHS[groups[0].lower()]
                    year = int(groups[3])
            elif len(groups) == 3:
                # Case: 21/06/2024 or 21-06-2024
                day = int(groups[0])
                month = int(groups[1])
                year = int(groups[2])
            else:
                continue

            return f"{day:02d}/{int(month):02d}/{year}"
    return "Date not found"

# Function to read CSV and apply date parsing
def process_csv(file_path):
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        headers = reader.fieldnames
        text_column = headers[0]  # Assume first column is the text column
        for row in reader:
            text = row[text_column]
            extracted = extract_date(text)
            print(f"Text: {text}")
            print(f"Extracted Date: {extracted}\n")

# Run the parser
process_csv("date_parser_testcases.csv")

Text: The event will take place on March 5, 2023.
Extracted Date: 05/03/2023

Text: Her birthday is on 07/08/1990.
Extracted Date: 07/08/1990

Text: The deadline is 2022-12-31.
Extracted Date: Date not found

Text: We met on 1st of January 2000.
Extracted Date: Date not found

Text: The concert is scheduled for 15th September, 2021.
Extracted Date: 15/09/2021

Text: Let's catch up on 02.04.2022.
Extracted Date: Date not found

Text: The project started on 5/6/19.
Extracted Date: Date not found

Text: He was born on 1987/11/23.
Extracted Date: Date not found

Text: Christmas is on 25th Dec 2024.
Extracted Date: Date not found

Text: The meeting is set for April 03, 2020.
Extracted Date: 03/04/2020

Text: Her birthdate, noted as 1997-05-20, is in the records.
Extracted Date: Date not found

Text: Her appointment is on the 2nd of March, 2021.
Extracted Date: Date not found

Text: The exam date is 2021.11.10.
Extracted Date: Date not found

Text: They got married on 12/12/12.
Extracted Dat

# **Part3**

In [ ]:
import csv
import re

# Pronoun mapping dictionaries
male_to_female = {
    "he": "she",
    "him": "her",
    "his": "her",
    "himself": "herself"
}

female_to_male = {
    "she": "he",
    "her": "him",
    "hers": "his",
    "herself": "himself"
}

# Words suggesting "her" is possessive
possessive_contexts = ["book", "car", "bag", "pen", "idea", "dog", "phone", "shoes", "attitude", "trip", "laptop"]

def is_possessive(word, next_word):
    return next_word and next_word.lower() in possessive_contexts

# Main transformation function
def transform_sentence(sentence, target_gender):
    words = sentence.split()
    transformed = []

    i = 0
    while i < len(words):
        word = words[i]
        stripped = re.sub(r'[^\w]', '', word)
        punct = word[len(stripped):] if len(stripped) < len(word) else ''

        lower = stripped.lower()
        next_word = words[i + 1] if i + 1 < len(words) else ""

        # Use correct mapping
        if target_gender == "female" and lower in male_to_female:
            if lower == "his":
                replacement = "her"
            else:
                replacement = male_to_female[lower]
        elif target_gender == "male" and lower in female_to_male:
            if lower == "her":
                replacement = "his" if is_possessive(lower, next_word) else "him"
            else:
                replacement = female_to_male[lower]
        else:
            replacement = stripped

        # Preserve case
        if stripped.istitle():
            replacement = replacement.capitalize()
        elif stripped.isupper():
            replacement = replacement.upper()

        transformed.append(replacement + punct)
        i += 1

    return ' '.join(transformed)

# Process CSV and check accuracy
def process_pronoun_csv(filepath):
    correct = 0
    total = 0

    with open(filepath, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            input_text = row['input_text']
            target_gender = row['target_gender'].strip().lower()
            expected = row['expected_output'].strip()

            output = transform_sentence(input_text, target_gender).strip()

            match = output.strip().lower() == expected.strip().lower()
            print(f"Input:    {input_text}")
            print(f"Gender:   {target_gender}")
            print(f"Output:   {output}")
            print(f"Expected: {expected}")
            print(f"Match:    {match}")
            print("-" * 50)

            if match:
                correct += 1
            total += 1

    print(f"\n✅ Accuracy: {correct}/{total} correct ({(correct / total) * 100:.2f}%)")

# Run
process_pronoun_csv("pronoun_testcases.csv")

Input:    He is going to the market.
Gender:   female
Output:   She is going to the market.
Expected: She is going to the market.
Match:    True
--------------------------------------------------
Input:    His book is on the table.
Gender:   female
Output:   Her book is on the table.
Expected: Her book is on the table.
Match:    True
--------------------------------------------------
Input:    I saw him yesterday.
Gender:   female
Output:   I saw her yesterday.
Expected: I saw her yesterday.
Match:    True
--------------------------------------------------
Input:    He hurt himself.
Gender:   female
Output:   She hurt herself.
Expected: She hurt herself.
Match:    True
--------------------------------------------------
Input:    I called him last night.
Gender:   female
Output:   I called her last night.
Expected: I called her last night.
Match:    True
--------------------------------------------------
Input:    That is his car.
Gender:   female
Output:   That is her car.
Expected: Th